In [ ]:
import numpy as np
import pandas as pd
import gdstk
import matplotlib.pyplot as plt
from cytoolz import unique

%matplotlib inline
from functools import partial

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.microfluidics_design as microfluidics_design
import paulssonlab.microfluidics_design.fluid as fluid
import paulssonlab.microfluidics_design.fov as fov

In [ ]:
import pint

u = pint._DEFAULT_REGISTRY

mu = 0.7 * u.cP  # dynamic viscosity
nu = mu / (1 * u.g / u.cm**3)  # kinematic viscosity

In [ ]:
camera_dims = {
    "iris15": (4.25, np.array([5056, 2960])),
    "iris9": (4.25, np.array([2960, 2960])),
    "bsi": (6.5, np.array([2048, 2048])),
}
mags = (20, 40)
fov_dims = {}
for camera, (pixel_size, fov_dim) in camera_dims.items():
    for mag in mags:
        fov_dims[f"{camera} {mag}x"] = fov_dim * pixel_size / mag

In [ ]:
def print_flow_info(height, flow_rate, mu, nu, metadata):
    for name, md in metadata.items():
        R_, extra = fluid.snake_resistance(
            height, md, length_unit=u.um, return_extra=True
        )
        R = R_ * mu
        # print('R', R.to(u.mbar * u.min / u.uL))
        delta_P = R * flow_rate
        print(f"{name}: {delta_P.to(u.bar)}")
        print(
            f"    flow rate: {flow_rate} ({flow_rate.to(u.milliliters/u.hour)}, {flow_rate.to(u.milliliters/u.day)})"
        )
        if "flow_nonuniformity" in extra:
            print(f"    flow nonuniformity: {extra['flow_nonuniformity']}")
        if "manifold_width" in md:
            L_e = fluid.entrance_length(
                height,
                md["manifold_width"] * u.um,
                flow_rate,
                nu,
            )
            print(f"    entrance length: {L_e}")

# Designs

In [ ]:
coverslip_dims = np.array([55e3, 24e3])
# chip_dims = np.array([33e3, 18e3])
chip_dims0 = np.array([33e3, 18e3])
# chip_dims0 = np.array([23e3, 13e3])
# chip_dims = np.array([38e3, 19e3])
chip_dims = np.array([35e3, 19e3])

In [ ]:
plt.figure(figsize=(10, 5))
ax = plt.gca()
ax.add_patch(
    plt.Rectangle(-coverslip_dims / 2, *coverslip_dims, fill=False, color="red", lw=2)
)
ax.add_patch(plt.Rectangle(-chip_dims0 / 2, *chip_dims0, fill=False, ls="-."))
# ax.add_patch(plt.Rectangle(-chip_dims1 / 2, *chip_dims1, fill=False, ls="--"))
ax.add_patch(plt.Rectangle(-chip_dims / 2, *chip_dims, fill=False, lw=2))
ax.set_xlim(-1.2 * coverslip_dims[0] / 2, 1.2 * coverslip_dims[0] / 2)
ax.set_ylim(-1.2 * coverslip_dims[1] / 2, 1.2 * coverslip_dims[1] / 2)
ax.set_aspect("equal")

In [ ]:
%%time
metadata = {}
base_params = dict(
    design_func=microfluidics_design.snake,
    dims=chip_dims,
    trench_length=35,
    feeding_channel_width=40,
    trench_gap=20,
    # trench_width=1.4,
    trench_spacing=2.1,
    trench_margin=0,
    border_margin=0.5e3,
    port_margin=1e3,
    port_radius=750 / 2,
    # port_radius=0,
    port=False,
    registration_marks="qr",
    registration_mark_barcodes=False,
    trenches=True,
    metadata=metadata,
)
manifold_params = {
    **base_params,
    **dict(
        design_func=microfluidics_design.manifold_snake,
        lanes_per_snake=3,
        manifold_input_style="bend-out",
        manifold_split=4,
        manifold_width=50,
        manifold_input_margin=775,  # 2e3,
        manifold_bend_margin=True,  # 0.2e3,
        manifold_bend_radius=0.1e3,
        manifold_margin=100,
    ),
}

wafer_id = 1
split_manifold_chip = dict(trench_width=1.4)
split_poor_manifold_chip = dict(
    manifold_trench_params=[
        dict(trench_width=1.3),
        dict(trench_width=1.3),
        dict(trench_width=1.5),
        dict(trench_width=1.5),
    ]
)

params = [split_manifold_chip, split_manifold_chip, split_poor_manifold_chip]
params = [
    {
        **manifold_params,
        **p,
        "ur_corner_label": f"{wafer_id}.{chip_id}",
    }
    for chip_id, p in enumerate(params, 1)
]
# chip names
chip_names = [
    "Basilisk {ports} FC{feeding_channel_width} L{trench_length} W{trench_width_} TS{trench_spacing}{reg} TG{trench_gap}".format(
        ports=f"S{p['split']}"
        if "split" in p and np.isscalar(p["split"])
        else "S{} LS{}".format(p["manifold_split"], p["lanes_per_snake"]),
        reg=f" M{p.get('mark_size')} ID{p.get('chip_id')}"
        if p.get("registration_mark_barcodes")
        else "",
        trench_width_=p["trench_width"]
        if "trench_width" in p
        else "/".join(
            unique([str(d["trench_width"]) for d in p["manifold_trench_params"]])
        ),
        **p,
    )
    for p in params
]
# make chip cells
chips = [microfluidics_design.chip(name, **p) for p, name in zip(params, chip_names)]
# calculate FOV info and overlay FOVs on chip cells
grid_metadata = {}
draw_grid = True
for chip_name, chip in zip(chip_names, chips):
    chip_grid_metadata = fov.get_grid_metadata(
        fov_dims, metadata[chip_name], skip_first=True
    )
    grid_metadata[chip_name] = chip_grid_metadata
    if draw_grid:
        fov.draw_grid_overlay(
            chip,
            metadata[chip_name],
            fov_dims,
            chip_grid_metadata,
            center_margins=False,
            rotate=False,
        )
grid_df = pd.concat(
    {
        k: pd.DataFrame(v.values()).set_index("fov_name")
        for k, v in grid_metadata.items()
    },
    names=["chip"],
)
# lay out wafer, output design
wafer_manifest = "\n".join([f"{idx+1}) " + name for idx, name in enumerate(chip_names)])
wafer_diameter = 3 * u.inch
alignment_mark_position = wafer_diameter / 2 - 0.2 * u.inch
main_cell = microfluidics_design.wafer(
    chips,
    label_right="Basilisk\n JQS/DE/YG 230328",
    label_left=wafer_manifest,
    label=True,
    mask=False,
    chip_dims=chip_dims,
    diameter=wafer_diameter.to(u.um).magnitude,
    alignment_mark_position=alignment_mark_position.to(u.um).magnitude,
)
MAX_POINTS = 2000  # LayoutEditor uses 8191, gdstk default is 199
microfluidics_design.write_gds(main_cell, "designs/230328basilisk.gds", max_points=MAX_POINTS)

In [ ]:
print(wafer_manifest)

In [ ]:
{k: v["trenches_per_input"] for k, v in metadata.items()}

In [ ]:
{k: v["num_trenches"] for k, v in metadata.items()}

In [ ]:
{k: v["lanes_per_input"] for k, v in metadata.items()}

In [ ]:
{k: v["num_lanes"] for k, v in metadata.items()}

In [ ]:
# Q = 300 * u.uL / u.min
Q = 75 * u.uL / u.min
height = 70 * u.um
print_flow_info(height, Q, mu, nu, metadata)

In [ ]:
grid_df.loc[pd.IndexSlice[:, ["iris15 20x", "iris9 20x", "bsi 40x"]], :].round(2)

# Fluids

# FOVs

In [ ]:
# skip_first -> offset_active_regions = 0, -1, +1 (in draw_fov_grid)
# give offset sequence for each graph cycle and each offset_active_regions

In [ ]:
# active, inactive, active, inactive...
trench_length = 35
feeding_channel_width = 40
trench_gap = 20
region_lengths = [trench_length, feeding_channel_width, trench_length, trench_gap]
num_regions = len(region_lengths)
fov = fov_dims["iris15 20x"][1]

In [ ]:
def next_transition(x0, region_lengths):
    num_regions = len(region_lengths)
    total_length = sum(region_lengths)
    x0_wraparound, x0 = divmod(x0, total_length)
    x = 0
    region = 0
    while True:
        x += region_lengths[region]
        if x > x0:
            return x + x0_wraparound * total_length, region
        region = (region + 1) % num_regions

In [ ]:
2960 * 4.25 / 20

In [ ]:
next_transition(35, region_lengths)

In [ ]:
def fov_positions(fov_height, region_lengths):
    total_length = sum(region_lengths)
    configs = []
    x = 0
    while x < total_length:
        new_top, top_idx = next_transition(x, region_lengths)
        new_bottom, bottom_idx = next_transition(x + fov_height, region_lengths)
        top_margin = new_top - x
        bottom_margin = new_bottom - (x + fov_height)
        if top_idx % 2 == 1 and bottom_idx % 2 == 1:
            configs.append(
                {
                    "top": x,
                    "bottom": x + fov_height,
                    "top_margin": top_margin,
                    "bottom_margin": bottom_margin,
                }
            )
        if top_margin < bottom_margin:
            x = new_top
        else:
            x = new_bottom - fov_height
    return configs

In [ ]:
(370 + 20) % total_length

In [ ]:
fov

In [ ]:
repeats = int((2 * fov) // total_length)
plt.figure(figsize=(12, 2))
plt.step(
    np.concatenate(((0, 0), np.cumsum(region_lengths * repeats))),
    np.concatenate(((0, 1), np.arange(len(region_lengths) * repeats) % 2 == 0)),
    where="pre",
)
y = 56
plt.plot([0, y, y, y + fov, y + fov], [0, 0, 0.5, 0.5, 0])

In [ ]:
for start_region in range(num_regions, 2):
    region = start_region
    position = 0
    while True:
        if position + region_lengths[region] <= fov:
            position += region_lengths[region]
            region = (region + 1) % num_regions
        else:
            fovs.append(dict(margin=))

In [ ]:
# start in every inactive region (even region)

In [ ]:
{"margin": 30, "start_region": 0, "end_region": 0}